#####About
<I>Extracts features from <b>TRAINING</b> files and saves as <b>X_train.npy</b> and <b>y_train.npy</b>

##Importing Libraries

In [1]:
# Importing Libraries 
import csv
import math
import pandas as pd
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Imputer,normalize
from sklearn import preprocessing
from pandas.stats.api import ols
import statsmodels.api as sm
from scipy.stats import linregress
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [40]:
# reading training files 
age=pd.read_csv('id_age_train.csv')
lbl=pd.read_csv('label_train.csv')
vit=pd.read_csv('vitals_train.csv', usecols=['ID','TIME','ICU'])
lab=pd.read_csv('id_time_labs_train.csv')

In [45]:
vit

,ID,TIME,ICU
0,1,0,0
1,1,4320,0
2,1,5646,1
3,1,5703,1
4,1,6342,1
5,1,6609,1
6,1,6894,1
7,1,6957,1
8,1,7511,1
9,1,8372,1


In [25]:
# df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': [1, 2, 3],
#                     'C': [16, 34, 12]})
# pd.get_dummies(df)

,B,C,A_a,A_b
0,1,16,1,0
1,2,34,0,1
2,3,12,1,0


In [101]:
# reading validation files
# age_val=pd.read_csv('id_age_val.csv')
# lab_val=pd.read_csv('id_time_labs_val.csv')
# vit_val=pd.read_csv('id_time_vitals_val.csv')


In [102]:
# combining Training and Validation dataframes for combined training
# age=pd.concat([age,age_val],ignore_index=True,axis=0)
# vit=pd.concat([vit,vit_val],ignore_index=True,axis=0)
# lab=pd.concat([lab,lab_val],ignore_index=True,axis=0)


In [103]:
vit=vit.fillna(vit.median())

In [104]:
vit[[u'V1', u'V2', u'V3', u'V4', u'V5', u'V6']]=vit[[u'V1', u'V2', u'V3', u'V4', u'V5', u'V6']].applymap(lambda x: math.log(1+x))


In [105]:
# filling labs values with ideal values

# values=[7.4,40,85,140,43.5,25,13,0.9,7,48,275,1.1,1400,190, 1.3,0.01,0.01,87.5,87.5, 47.39,4.4,94,25,50,2]
# lab2=lab.copy()
# for i,col in enumerate(lab.columns[2:]):
#     lab[col]=lab[col].fillna()
#     lab2[col]=values[i] 
    
# result=(lab-lab2)
# result['ID']=lab.ID
# result['TIME']=lab.TIME
# lab=result.copy()
lab=lab.fillna(lab.median())
lab=lab.applymap(lambda x: math.log(1+x))

## Feature Extraction

In [106]:
# y - LABELS
y_train=np.array(lbl.LABEL.values)
# y_val=[]
# lbl_val=open('id_label_val.txt','rb')
# for f in lbl_val.readlines():
#     y_val.append( f.split(',')[1].strip())
# y_train=np.concatenate([y_train,y_val])
np.save('y_train.npy',y_train)
len(y_train)

3594

In [107]:
#  Extracting features from VITALS

vit_m=vit
# stdev=vit_m.groupby(by='ID').std() # std deviation
med=vit_m.groupby(by='ID').median()  # median 
maxm=vit_m.groupby(by='ID').max() 
minm=vit_m.groupby(by='ID').min() 
max_min=maxm-minm  # maximum - minimum
num_stamps=vit_m.groupby(by='ID').count()

t_diff=vit_m.groupby(by='ID').apply(lambda x: (x.iat[len(x)-1,1])/len(x))
t_diff_arr=[[item] for item in t_diff.values]


In [108]:
#  Features from LABS
med_lab=lab.groupby(by='ID').median()  # median 
maxm_lab=lab.groupby(by='ID').max() 
minm_lab=lab.groupby(by='ID').min()
max_min_lab=maxm_lab-minm_lab  # maximum - minimum

In [109]:
# imp=Imputer(missing_values='NaN',strategy='mean')
# stdev=imp.fit_transform(stdev)

In [110]:
first_values=[]
for i,column in enumerate(['V1','V2','V3','V4','V5','V6']):
    ser=vit_m[column][vit_m.groupby(by='ID')[column].apply(lambda x: x.first_valid_index())].values
    first_values.append(ser)
first_values=np.array(first_values).transpose()  # contains first values of all vitals for each patient


In [111]:
last_values=[]
for i,column in enumerate(['V1','V2','V3','V4','V5','V6']):
    ser=vit_m[column][vit_m.groupby(by='ID')[column].apply(lambda x: x.last_valid_index())].values
    last_values.append(ser)
last_values=np.array(last_values).transpose()  # contains last values of all vitals for each patient


## Combining extracted features from different dataframes 

In [112]:

feat=np.concatenate([age.values[:,1:2],
                    med.iloc[:,1:7].values,
                    maxm.iloc[:,1:7].values, 
                    minm.iloc[:,1:7].values, 
                    max_min.iloc[:,1:7].values,
                    num_stamps.values[:,0:1],
                    t_diff_arr]
                    ,axis=1)
#                 no_values.iloc[:,2:8].values],axis=1) 

In [113]:
lab_features=np.concatenate([
                            med_lab.iloc[:,1:26].values,
                            maxm_lab.iloc[:,1:26].values,
                            minm_lab.iloc[:,1:26].values,
                            max_min_lab.iloc[:,1:26].values
                            ],axis=1)
        
                            

In [114]:
x_train=np.concatenate([feat, 
                        first_values, 
                        last_values ,
                        lab_features
                        ],axis=1)

In [115]:
np.save('x_train.npy',x_train)
x_train.shape

(3594L, 139L)